In [1]:
#| default_exp attribution

In [2]:
#| export
import os
os.chdir("/project/validating_attribution_techniques/torchCAM")
from torchcam.methods.gradient import SmoothGradCAMpp, GradCAMpp, GradCAM, LayerCAM
from torchcam.methods.activation import ScoreCAM
from torchcam.utils import overlay_mask
from torchvision.models import resnet50, resnet18, vgg16, inception_v3



In [3]:
#| export
os.chdir("/project/validating_attribution_techniques/")
from commons.api.viz import visualize_saliency_overlay
from commons.api.imagenet_tiny import get_imagenet_classes, get_random_images

import numpy as np
import matplotlib.pyplot as plt
import torch

import cv2


In [4]:
#| export
classes = get_imagenet_classes()

In [5]:
#| export
def attribution(model, input_tensor, method, target_layer = "layer4"):
    """
    Function used to generate a saliency map using different saliency generation techniques like GradCAM, ScoreCAM, SmoothGradCAMpp, LayerCAM.

    Args:
        model (torchvision.models): Model used in finding the class from an image.
            Example: ResNet50
            Import the model: from torchvision.models import resnet50
            Create an instance: model_resnet50 = resnet50(pretrained=True).eval()

        input_tensor (torch.Tensor): An input image transformed into a tensor with shape (3, w, h),
            where w is the width and h is the height.

        method (str): Name of the saliency generation method.

        target_layer (str or torch.nn.Module): The target layer for feature extraction. You can provide either the layer name as a string (e.g., "layer4") or the actual layer from the model.

    Returns:
        cam_mask (numpy.ndarray): The saliency map generated by the specified method.
        idx (int): The index of the predicted class with the highest probability.
        probs (numpy.ndarray): Class probabilities for the input image.

    Usage:
    - Call this function to generate a saliency map for the given input image using the specified method.
    - The target layer can be provided as either a string or the actual layer from the model.
    """
    def method1(method, model, target_layer):
        """
        Factory function to create an instance of a saliency method for a given model and target layer.
    
        Args:
            method (str): The name of the saliency generation method.
            model (torchvision.models): The model for which the saliency method instance is created.
            target_layer (str or torch.nn.Module): The target layer for feature extraction.
    
        Returns:
            torchcam.CAM: An instance of the specified saliency method for the given model and target layer.
    
        Usage:
        - Call this function to create an instance of a saliency method based on the method name and target layer.
        """
        if method == "ScoreCAM":
            return ScoreCAM(model, target_layer)
        elif method == "GradCAM":
            return GradCAM(model, target_layer)
        elif method == "SmoothGradCAMpp":
            return SmoothGradCAMpp(model, target_layer)
        elif method == "LayerCAM":
            return LayerCAM(model, target_layer)
            
    attr = method1(method, model, target_layer)
    with attr as extractor:  
        # Preprocess your data and feed it to the model
        out = model(input_tensor.unsqueeze(0))
        # Retrieve the CAM by passing the class index and the model output
        activation_map = extractor(out.squeeze(0).argmax().item(), out)
    # Apply softmax to calculate class probabilities
    probs = torch.softmax(out, dim=1)
    idx = probs.argmax()
    
    # Get the class with the highest probability for each input
    pred_class = torch.argmax(probs, dim=1)
    probs = probs.detach().cpu().numpy()
    
    # To extract the class ID and probability for the first input (if it's a batch):
    class_id = pred_class[0].item()
    probability = probs[0, class_id].item()
    cam_mask =  activation_map[0].cpu().numpy().squeeze(0)
    return cam_mask, idx, probs

In [8]:
#| hide
# os.chdir("/project/validating_attribution_techniques/shardul/api_notebooks/")
from nbdev.export import nb_export
nb_export('/project/validating_attribution_techniques/shardul/api_notebooks/attribution.ipynb', '/project/validating_attribution_techniques/commons/api/method/')